### Importing libraries 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
import ast

### Loading the dataset

In [2]:
expenditures_df = pd.read_csv('../SavedData/dataset2_expenditures.csv')
payments_df= pd.read_csv('../SavedData/dataset2_payments.csv')
expenditures_users_df = pd.read_csv('../SavedData/dataset2_expenditures_by_users.csv')
payments_users_df = pd.read_csv('../SavedData/dataset2_payments_by_users.csv')
simulated_cleaned_tx_df = pd.read_csv('../SavedData/dataset2_cleaned.csv')

In [3]:
simulated_cleaned_tx_df

,Date,Timestamp,Account No,Balance,Amount,Third Party Account No,Third Party Name,Datetime
0,01/01/2023,00:00:00,678330503.0,2971.000000,1584.00,NaN,Westport Care Home,2023-01-01 00:00:00
1,01/01/2023,00:00:00,472213568.0,3792.000000,1950.00,NaN,Barbiee Boutique,2023-01-01 00:00:00
2,01/01/2023,00:00:00,472213568.0,3012.000000,-780.00,283027736.0,NaN,2023-01-01 00:00:00
3,01/01/2023,00:00:00,283027736.0,1787.000000,780.00,472213568.0,NaN,2023-01-01 00:00:00
4,01/01/2023,00:00:00,624500124.0,3226.000000,1825.00,NaN,Fat Face,2023-01-01 00:00:00
...,...,...,...,...,...,...,...,...
229189,06/12/2023,20:54:00,581655972.0,45935.206861,-41.06,NaN,Tesco,2023-12-06 20:54:00
229190,06/12/2023,20:55:00,786141370.0,-244.837500,-62.35,NaN,Sainsbury Local,2023-12-06 20:55:00
229191,06/12/2023,21:05:00,824916823.0,9709.172159,-32.94,NaN,Deliveroo,2023-12-06 21:05:00
229192,06/12/2023,21:13:00,366550080.0,26834.165794,-19.25,NaN,Amazon,2023-12-06 21:13:00


In [87]:
expenditures_users_df

,Account No,total_transactions,total_spent,median_transaction,mean_transaction,business_interactions,user_interactions,most_recent_transaction
0,101531259.0,110,17627.58,88.580,160.250727,"['LBG', 'Disney', 'Grand Union BJJ', 'Matalan'...",[],2023-12-05
1,104832000.0,150,25115.52,49.920,167.436800,"['LBG', 'Netflix', 'Grand Union BJJ', 'Blizzar...",[587923845.0],2023-12-03
2,105375973.0,146,10067.28,48.160,68.953973,"['PureGym', 'Netflix', 'Amazon', ""Blackwell's""...",[],2023-12-03
3,106601471.0,180,7760.01,19.965,43.111167,"['Mojang Studios', 'Netflix', 'The Crown', ""Bl...",[],2023-12-06
4,108481285.0,344,15062.28,21.510,43.785698,"['Disney', 'Amazon', 'PureGym', ""Blackwell's"",...",[913801979.0],2023-12-06
...,...,...,...,...,...,...,...,...
971,995615876.0,172,17598.80,52.920,102.318605,"['LBG', 'Blizzard', 'Mojang Studios', 'The Cro...",[],2023-12-05
972,996042490.0,243,35268.01,34.970,145.135844,"['Halifax', 'Netflix', 'PureGym', ""Blackwell's...",[792845450.0],2023-12-06
973,998390769.0,140,21323.35,69.990,152.309643,"['Halifax', 'Disney', 'Netflix', 'Mojang Studi...",[],2023-12-01
974,998405607.0,145,8816.16,35.070,60.801103,"['Xbox', 'Blizzard', 'Mojang Studios', 'Netfli...",[],2023-11-30


### Splitting the data into training and testing 

In [4]:
copied_simulated_tx_df = simulated_cleaned_tx_df.copy()

# Splitting the copied dataset into historical_transactions (95%) and testing_transactions (5%)
training_transactions, testing_transactions = train_test_split(copied_simulated_tx_df, test_size=0.1, random_state=42)

# Displaying the shapes to verify the split
training_transactions.shape, testing_transactions.shape


((206274, 8), (22920, 8))

In [5]:
testing_transactions

,Date,Timestamp,Account No,Balance,Amount,Third Party Account No,Third Party Name,Datetime
110276,26/06/2023,05:13:00,547010927.0,14786.249167,-7.74,NaN,Amazon,2023-06-26 05:13:00
113678,01/07/2023,00:00:00,986774156.0,1666.714362,-939.00,NaN,LBG,2023-07-01 00:00:00
102644,09/06/2023,03:36:00,183888402.0,13918.517600,-36.25,NaN,JustEat,2023-06-09 03:36:00
98481,01/06/2023,08:12:00,822718326.0,6732.194082,-9.13,NaN,Tesco,2023-06-01 08:12:00
135544,31/07/2023,23:59:00,494156986.0,-312.797554,-7.99,NaN,Disney,2023-07-31 23:59:00
...,...,...,...,...,...,...,...,...
55831,30/03/2023,12:53:00,635679314.0,-976.793829,-96.68,NaN,Tesco,2023-03-30 12:53:00
188677,14/10/2023,10:39:00,421881811.0,3080.374643,-36.11,NaN,Brilliant Brushes,2023-10-14 10:39:00
96461,31/05/2023,23:59:00,593191200.0,1764.992250,-18.99,NaN,PureGym,2023-05-31 23:59:00
182205,02/10/2023,12:43:00,165691047.0,98.783000,-148.00,NaN,Matalan,2023-10-02 12:43:00


### Splitting the training transactions into payments and expenditures

In [6]:
payments_training_df = training_transactions[training_transactions['Amount'] > 0]
expenditures_training_df = training_transactions[training_transactions['Amount'] < 0]

In [7]:
# Taking the absolute values of the amount column
expenditures_training_df["Amount"] = expenditures_training_df["Amount"].abs()

expenditures_training_df

/var/folders/rw/215myrns4md_n1k8nkyww6mw0000gn/T/ipykernel_39856/721076894.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  expenditures_training_df["Amount"] = expenditures_training_df["Amount"].abs()


,Date,Timestamp,Account No,Balance,Amount,Third Party Account No,Third Party Name,Datetime
108236,21/06/2023,09:03:00,809499628.0,10775.476914,48.10,NaN,Tesco,2023-06-21 09:03:00
204684,02/11/2023,19:22:00,518474507.0,15455.325500,29.19,NaN,Deliveroo,2023-11-02 19:22:00
19823,30/01/2023,11:29:00,655368335.0,2230.145000,99.98,NaN,Gamestation,2023-01-30 11:29:00
151016,25/08/2023,11:09:00,678851253.0,4313.628611,63.70,NaN,Fat Face,2023-08-25 11:09:00
26217,31/01/2023,23:59:00,163470640.0,60.014100,12.99,NaN,SquareOnix,2023-01-31 23:59:00
...,...,...,...,...,...,...,...,...
119879,03/07/2023,18:54:00,424397946.0,2450.930784,119.62,NaN,Sainsbury,2023-07-03 18:54:00
103694,11/06/2023,10:27:00,241459011.0,995.080013,15.98,NaN,Sports Direct,2023-06-11 10:27:00
131932,29/07/2023,14:02:00,744211756.0,-3127.708250,44.72,NaN,Tesco,2023-07-29 14:02:00
146867,16/08/2023,17:57:00,245690309.0,7640.773332,46.45,NaN,Tesco,2023-08-16 17:57:00


### Feature engineering for anomaly detection: expenditure

In [8]:
# Calculating Q1, Q3, and IQR for each account in 'from_totally_fake_account'
quartiles = expenditures_training_df.groupby('Account No')['Amount'].quantile([0.25, 0.5, 0.75]).unstack()
quartiles['IQR'] = quartiles[0.75] - quartiles[0.25]

thresholds = quartiles.copy()
thresholds['Overall_Threshold_1_IQR'] = quartiles[0.75] + 1.5 * quartiles['IQR']
thresholds['Overall_Threshold_2_IQR'] = quartiles[0.75] + 3 * quartiles['IQR']
thresholds['Overall_Threshold_3_IQR'] = quartiles[0.75] + 5 * quartiles['IQR']

anomaly_detection_by_expenditures_df = thresholds


anomaly_detection_by_expenditures_df

,0.25,0.5,0.75,IQR,Overall_Threshold_1_IQR,Overall_Threshold_2_IQR,Overall_Threshold_3_IQR
Account No,,,,,,,
101531259.0,25.2200,78.000,120.8750,95.6550,264.35750,407.8400,599.1500
104832000.0,16.5125,49.785,106.7800,90.2675,242.18125,377.5825,558.1175
105375973.0,18.9900,43.725,102.3650,83.3750,227.42750,352.4900,519.2400
106601471.0,10.6900,18.705,45.5975,34.9075,97.95875,150.3200,220.1350
108481285.0,10.3350,21.980,45.6275,35.2925,98.56625,151.5050,222.0900
...,...,...,...,...,...,...,...
995615876.0,25.8750,53.850,108.9750,83.1000,233.62500,358.2750,524.4750
996042490.0,15.9900,34.730,93.4750,77.4850,209.70250,325.9300,480.9000
998390769.0,15.5600,74.790,145.4825,129.9225,340.36625,535.2500,795.0950


### Creating thresholds for each user to user transfers

In [9]:
def calculate_thresholds(group, multiplier):
    if len(group) <= 1:  # Check if there's only one expenditure
        return np.nan  # Indicative value for groups with only one transaction
    quantiles = group['Amount'].quantile([0.25, 0.75])
    IQR = quantiles[0.75] - quantiles[0.25]
    threshold = quantiles[0.75] + multiplier * IQR
    return threshold

# Applying the adjusted function for each threshold calculation with different multipliers

# For Threshold 1 (1.5 * IQR)
thresholds_1 = expenditures_training_df.dropna(subset=['Account No', 'Third Party Account No']).groupby(['Account No', 'Third Party Account No']).apply(lambda group: calculate_thresholds(group, 1.5)).reset_index(name='Threshold_1')
thresholds_structured_1 = thresholds_1.groupby('Account No').apply(lambda x: dict(zip(x['Third Party Account No'], x['Threshold_1']))).to_dict()

# For Threshold 2 (3 * IQR)
thresholds_2 = expenditures_training_df.dropna(subset=['Account No', 'Third Party Account No']).groupby(['Account No', 'Third Party Account No']).apply(lambda group: calculate_thresholds(group, 3)).reset_index(name='Threshold_2')
thresholds_structured_2 = thresholds_2.groupby('Account No').apply(lambda x: dict(zip(x['Third Party Account No'], x['Threshold_2']))).to_dict()

# For Threshold 3 (5 * IQR)
thresholds_3 = expenditures_training_df.dropna(subset=['Account No', 'Third Party Account No']).groupby(['Account No', 'Third Party Account No']).apply(lambda group: calculate_thresholds(group, 5)).reset_index(name='Threshold_3')
thresholds_structured_3 = thresholds_3.groupby('Account No').apply(lambda x: dict(zip(x['Third Party Account No'], x['Threshold_3']))).to_dict()

anomaly_detection_by_expenditures_df['Third_Party_Account_Threshold_1'] = anomaly_detection_by_expenditures_df.index.map(lambda x: thresholds_structured_1.get(x, np.nan))
anomaly_detection_by_expenditures_df['Third_Party_Account_Threshold_2'] = anomaly_detection_by_expenditures_df.index.map(lambda x: thresholds_structured_2.get(x, np.nan))
anomaly_detection_by_expenditures_df['Third_Party_Account_Threshold_3'] = anomaly_detection_by_expenditures_df.index.map(lambda x: thresholds_structured_3.get(x, np.nan))

In [10]:
anomaly_detection_by_expenditures_df

,0.25,0.5,0.75,IQR,Overall_Threshold_1_IQR,Overall_Threshold_2_IQR,Overall_Threshold_3_IQR,Third_Party_Account_Threshold_1,Third_Party_Account_Threshold_2,Third_Party_Account_Threshold_3
Account No,,,,,,,,,,
101531259.0,25.2200,78.000,120.8750,95.6550,264.35750,407.8400,599.1500,NaN,NaN,NaN
104832000.0,16.5125,49.785,106.7800,90.2675,242.18125,377.5825,558.1175,{587923845.0: 670.0},{587923845.0: 670.0},{587923845.0: 670.0}
105375973.0,18.9900,43.725,102.3650,83.3750,227.42750,352.4900,519.2400,NaN,NaN,NaN
106601471.0,10.6900,18.705,45.5975,34.9075,97.95875,150.3200,220.1350,NaN,NaN,NaN
108481285.0,10.3350,21.980,45.6275,35.2925,98.56625,151.5050,222.0900,{913801979.0: 2185.0},{913801979.0: 2552.5},{913801979.0: 3042.5}
...,...,...,...,...,...,...,...,...,...,...
995615876.0,25.8750,53.850,108.9750,83.1000,233.62500,358.2750,524.4750,NaN,NaN,NaN
996042490.0,15.9900,34.730,93.4750,77.4850,209.70250,325.9300,480.9000,{792845450.0: 1500.0},{792845450.0: 1500.0},{792845450.0: 1500.0}
998390769.0,15.5600,74.790,145.4825,129.9225,340.36625,535.2500,795.0950,NaN,NaN,NaN


### Creating thresholds for each user to industry

In [11]:
industry_categories_expenditure = {
    "Coffee": ["Coffee #1", "Costa Coffee", "Starbucks"],
    "Technology and Gaming": ["Mojang Studios", "Xbox", "Blizzard", "SquareOnix"],
    "Entertainment and Media": ["Netflix", "Disney", "Amazon"],
    "Retail - Crafts, Hobbies and Books": ["Wool", "Etsy", "Hobby Lobby", "Cass Art", "Craftastic", "Hobbycraft", "The Works", "Blackwell's", "Brilliant Brushes", "Five Senses Art", "A Yarn Story", "Foyles", "AMAZON", "Stitch By Stitch", "Fitted Stitch"],
    "Retail - Clothing and Fashion": ["Reebok", "Sports Direct", "JD Sports", "Tesco", "Matalan", "Fat Face", "Topshop", "Revella", "Mountain Warehouse", "Millets", "Loosely Fitted"],
    "Retail - General and Department Stores": ["Coop Local", "Sainsbury Local", "Sainsbury", "Selfridges"],
    "Retail - High end": ["Barbiee Boutique", "North Face", "Collector Cave"],
    "Retail- Kids": ["Mamas & Papas", "Mothercare", "Gap Kids"],
    "Retail - services ": ["Vision Express", "Pets Corner", "Head", "A Cut Above", "Specsavers"],
    "Health and Fitness": ["PureGym", "Grand Union BJJ"],
    "Pharmacy": ["Boots", "Lloyds Pharmacy"],
    "Hospital and Care Home": ["University College Hospital", "Happy Days Home", "Remedy plus care"],
    "Electronics and Entertainment Retail": ["CeX", "Gamestation", "HMV"],
    "Child Care and Education": ["Lavender Primary", "Town High", "RugbyFields"],
    "Care Services": ["Happy Days Home", "Remedy plus care", "Kew House"],
    "Financial Services": ["LBG", "Halifax"],
    "Pubs": ["Kings Arms", "The Crown", "Rose & Crown"],
    "Takeaway": ["Deliveroo", "JustEat"],
    "Restaurant": ["Frankie & Bennies"],
    "Park": ["Victoria Park"]
}

In [12]:
# Add industry column to tx_df

def tx_df_industry_mapping(name):
    if isinstance(name, str):  # Check if input is a string
        for category, names_list in industry_categories_expenditure.items():
            if any(n in name for n in names_list):
                return category
    return np.nan


# Apply the mapping
expenditures_training_df['industry_category'] = expenditures_training_df['Third Party Name'].apply(tx_df_industry_mapping)

testing_transactions['industry_category'] = testing_transactions['Third Party Name'].apply(tx_df_industry_mapping)
testing_transactions

/var/folders/rw/215myrns4md_n1k8nkyww6mw0000gn/T/ipykernel_39856/3849273354.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  expenditures_training_df['industry_category'] = expenditures_training_df['Third Party Name'].apply(tx_df_industry_mapping)


,Date,Timestamp,Account No,Balance,Amount,Third Party Account No,Third Party Name,Datetime,industry_category
110276,26/06/2023,05:13:00,547010927.0,14786.249167,-7.74,NaN,Amazon,2023-06-26 05:13:00,Entertainment and Media
113678,01/07/2023,00:00:00,986774156.0,1666.714362,-939.00,NaN,LBG,2023-07-01 00:00:00,Financial Services
102644,09/06/2023,03:36:00,183888402.0,13918.517600,-36.25,NaN,JustEat,2023-06-09 03:36:00,Takeaway
98481,01/06/2023,08:12:00,822718326.0,6732.194082,-9.13,NaN,Tesco,2023-06-01 08:12:00,Retail - Clothing and Fashion
135544,31/07/2023,23:59:00,494156986.0,-312.797554,-7.99,NaN,Disney,2023-07-31 23:59:00,Entertainment and Media
...,...,...,...,...,...,...,...,...,...
55831,30/03/2023,12:53:00,635679314.0,-976.793829,-96.68,NaN,Tesco,2023-03-30 12:53:00,Retail - Clothing and Fashion
188677,14/10/2023,10:39:00,421881811.0,3080.374643,-36.11,NaN,Brilliant Brushes,2023-10-14 10:39:00,"Retail - Crafts, Hobbies and Books"
96461,31/05/2023,23:59:00,593191200.0,1764.992250,-18.99,NaN,PureGym,2023-05-31 23:59:00,Health and Fitness
182205,02/10/2023,12:43:00,165691047.0,98.783000,-148.00,NaN,Matalan,2023-10-02 12:43:00,Retail - Clothing and Fashion


In [13]:
# Re-define the calculate_thresholds function in case it's not defined
def calculate_thresholds(group, multiplier):
    quantiles = group['Amount'].quantile([0.25, 0.75])
    IQR = quantiles[0.75] - quantiles[0.25]
    threshold = quantiles[0.75] + multiplier * IQR
    return threshold

# Assuming business_accounts_df is already defined
grouped_by_industry = expenditures_training_df.groupby(['Account No', 'industry_category'])


# Calculate thresholds for each grouping, excluding groups with only one record
thresholds_structured_by_industry_and_category = {}
for (acc, cat), group in grouped_by_industry:
    if len(group) > 1:  # Check if group size is greater than 1
        thresholds_structured_by_industry_and_category[(acc, cat)] = {
            'threshold_1_IQR': calculate_thresholds(group, 1.5),
            'threshold_2_IQR': calculate_thresholds(group, 3),
            'threshold_3_IQR': calculate_thresholds(group, 5)
        }


# Create a mapping function that retrieves threshold values for a given user and industry
def map_thresholds_to_user(user_id, threshold_key):
    return {cat: details[threshold_key] for (acc, cat), details in thresholds_structured_by_industry_and_category.items() if acc == user_id}

# Convert the index to a regular column - Ensure this step is needed for your workflow
anomaly_detection_by_expenditures_df.reset_index(inplace=True)

# Apply the mapping function for each threshold level
anomaly_detection_by_expenditures_df['industry_threshold_1_IQR'] = anomaly_detection_by_expenditures_df['Account No'].apply(lambda x: map_thresholds_to_user(x, 'threshold_1_IQR'))
anomaly_detection_by_expenditures_df['industry_threshold_2_IQR'] = anomaly_detection_by_expenditures_df['Account No'].apply(lambda x: map_thresholds_to_user(x, 'threshold_2_IQR'))
anomaly_detection_by_expenditures_df['industry_threshold_3_IQR'] = anomaly_detection_by_expenditures_df['Account No'].apply(lambda x: map_thresholds_to_user(x, 'threshold_3_IQR'))

anomaly_detection_by_expenditures_df

,Account No,0.25,0.5,0.75,IQR,Overall_Threshold_1_IQR,Overall_Threshold_2_IQR,Overall_Threshold_3_IQR,Third_Party_Account_Threshold_1,Third_Party_Account_Threshold_2,Third_Party_Account_Threshold_3,industry_threshold_1_IQR,industry_threshold_2_IQR,industry_threshold_3_IQR
0,101531259.0,25.2200,78.000,120.8750,95.6550,264.35750,407.8400,599.1500,NaN,NaN,NaN,"{'Entertainment and Media': 7.99, 'Financial S...","{'Entertainment and Media': 7.99, 'Financial S...","{'Entertainment and Media': 7.99, 'Financial S..."
1,104832000.0,16.5125,49.785,106.7800,90.2675,242.18125,377.5825,558.1175,{587923845.0: 670.0},{587923845.0: 670.0},{587923845.0: 670.0},{'Electronics and Entertainment Retail': 189.9...,{'Electronics and Entertainment Retail': 272.4...,{'Electronics and Entertainment Retail': 382.3...
2,105375973.0,18.9900,43.725,102.3650,83.3750,227.42750,352.4900,519.2400,NaN,NaN,NaN,{'Entertainment and Media': 27.990000000000002...,"{'Entertainment and Media': 39.99, 'Health and...","{'Entertainment and Media': 55.99, 'Health and..."
3,106601471.0,10.6900,18.705,45.5975,34.9075,97.95875,150.3200,220.1350,NaN,NaN,NaN,{'Electronics and Entertainment Retail': 178.7...,{'Electronics and Entertainment Retail': 234.9...,{'Electronics and Entertainment Retail': 309.9...
4,108481285.0,10.3350,21.980,45.6275,35.2925,98.56625,151.5050,222.0900,{913801979.0: 2185.0},{913801979.0: 2552.5},{913801979.0: 3042.5},"{'Entertainment and Media': 29.73375, 'Health ...","{'Entertainment and Media': 42.78, 'Health and...","{'Entertainment and Media': 60.175, 'Health an..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,995615876.0,25.8750,53.850,108.9750,83.1000,233.62500,358.2750,524.4750,NaN,NaN,NaN,{'Electronics and Entertainment Retail': 126.2...,{'Electronics and Entertainment Retail': 174.9...,{'Electronics and Entertainment Retail': 239.9...
972,996042490.0,15.9900,34.730,93.4750,77.4850,209.70250,325.9300,480.9000,{792845450.0: 1500.0},{792845450.0: 1500.0},{792845450.0: 1500.0},{'Electronics and Entertainment Retail': 237.4...,{'Electronics and Entertainment Retail': 349.9...,{'Electronics and Entertainment Retail': 499.8...
973,998390769.0,15.5600,74.790,145.4825,129.9225,340.36625,535.2500,795.0950,NaN,NaN,NaN,{'Electronics and Entertainment Retail': 219.9...,{'Electronics and Entertainment Retail': 309.9...,{'Electronics and Entertainment Retail': 429.9...
974,998405607.0,15.9900,35.000,100.0000,84.0100,226.01500,352.0300,520.0500,NaN,NaN,NaN,{'Electronics and Entertainment Retail': 154.9...,{'Electronics and Entertainment Retail': 192.4...,{'Electronics and Entertainment Retail': 242.4...


In [14]:
anomaly_detection_by_expenditures_df

,Account No,0.25,0.5,0.75,IQR,Overall_Threshold_1_IQR,Overall_Threshold_2_IQR,Overall_Threshold_3_IQR,Third_Party_Account_Threshold_1,Third_Party_Account_Threshold_2,Third_Party_Account_Threshold_3,industry_threshold_1_IQR,industry_threshold_2_IQR,industry_threshold_3_IQR
0,101531259.0,25.2200,78.000,120.8750,95.6550,264.35750,407.8400,599.1500,NaN,NaN,NaN,"{'Entertainment and Media': 7.99, 'Financial S...","{'Entertainment and Media': 7.99, 'Financial S...","{'Entertainment and Media': 7.99, 'Financial S..."
1,104832000.0,16.5125,49.785,106.7800,90.2675,242.18125,377.5825,558.1175,{587923845.0: 670.0},{587923845.0: 670.0},{587923845.0: 670.0},{'Electronics and Entertainment Retail': 189.9...,{'Electronics and Entertainment Retail': 272.4...,{'Electronics and Entertainment Retail': 382.3...
2,105375973.0,18.9900,43.725,102.3650,83.3750,227.42750,352.4900,519.2400,NaN,NaN,NaN,{'Entertainment and Media': 27.990000000000002...,"{'Entertainment and Media': 39.99, 'Health and...","{'Entertainment and Media': 55.99, 'Health and..."
3,106601471.0,10.6900,18.705,45.5975,34.9075,97.95875,150.3200,220.1350,NaN,NaN,NaN,{'Electronics and Entertainment Retail': 178.7...,{'Electronics and Entertainment Retail': 234.9...,{'Electronics and Entertainment Retail': 309.9...
4,108481285.0,10.3350,21.980,45.6275,35.2925,98.56625,151.5050,222.0900,{913801979.0: 2185.0},{913801979.0: 2552.5},{913801979.0: 3042.5},"{'Entertainment and Media': 29.73375, 'Health ...","{'Entertainment and Media': 42.78, 'Health and...","{'Entertainment and Media': 60.175, 'Health an..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,995615876.0,25.8750,53.850,108.9750,83.1000,233.62500,358.2750,524.4750,NaN,NaN,NaN,{'Electronics and Entertainment Retail': 126.2...,{'Electronics and Entertainment Retail': 174.9...,{'Electronics and Entertainment Retail': 239.9...
972,996042490.0,15.9900,34.730,93.4750,77.4850,209.70250,325.9300,480.9000,{792845450.0: 1500.0},{792845450.0: 1500.0},{792845450.0: 1500.0},{'Electronics and Entertainment Retail': 237.4...,{'Electronics and Entertainment Retail': 349.9...,{'Electronics and Entertainment Retail': 499.8...
973,998390769.0,15.5600,74.790,145.4825,129.9225,340.36625,535.2500,795.0950,NaN,NaN,NaN,{'Electronics and Entertainment Retail': 219.9...,{'Electronics and Entertainment Retail': 309.9...,{'Electronics and Entertainment Retail': 429.9...
974,998405607.0,15.9900,35.000,100.0000,84.0100,226.01500,352.0300,520.0500,NaN,NaN,NaN,{'Electronics and Entertainment Retail': 154.9...,{'Electronics and Entertainment Retail': 192.4...,{'Electronics and Entertainment Retail': 242.4...


In [15]:
# Extracting keys from 'transfer_to_accounts_threshold_1_IQR' to create 'user_user_interaction'
anomaly_detection_by_expenditures_df['user_user_interaction'] = anomaly_detection_by_expenditures_df['Third_Party_Account_Threshold_1'].apply(lambda x: list(x.keys()) if isinstance(x, dict) else None)

# Extracting keys from 'industry_threshold_1_IQR' to create 'user_industry_interaction'
anomaly_detection_by_expenditures_df['user_industry_interaction'] = anomaly_detection_by_expenditures_df['industry_threshold_1_IQR'].apply(lambda x: list(x.keys()) if isinstance(x, dict) else None)

anomaly_detection_by_expenditures_df


,Account No,0.25,0.5,0.75,IQR,Overall_Threshold_1_IQR,Overall_Threshold_2_IQR,Overall_Threshold_3_IQR,Third_Party_Account_Threshold_1,Third_Party_Account_Threshold_2,Third_Party_Account_Threshold_3,industry_threshold_1_IQR,industry_threshold_2_IQR,industry_threshold_3_IQR,user_user_interaction,user_industry_interaction
0,101531259.0,25.2200,78.000,120.8750,95.6550,264.35750,407.8400,599.1500,NaN,NaN,NaN,"{'Entertainment and Media': 7.99, 'Financial S...","{'Entertainment and Media': 7.99, 'Financial S...","{'Entertainment and Media': 7.99, 'Financial S...",None,"[Entertainment and Media, Financial Services, ..."
1,104832000.0,16.5125,49.785,106.7800,90.2675,242.18125,377.5825,558.1175,{587923845.0: 670.0},{587923845.0: 670.0},{587923845.0: 670.0},{'Electronics and Entertainment Retail': 189.9...,{'Electronics and Entertainment Retail': 272.4...,{'Electronics and Entertainment Retail': 382.3...,[587923845.0],"[Electronics and Entertainment Retail, Enterta..."
2,105375973.0,18.9900,43.725,102.3650,83.3750,227.42750,352.4900,519.2400,NaN,NaN,NaN,{'Entertainment and Media': 27.990000000000002...,"{'Entertainment and Media': 39.99, 'Health and...","{'Entertainment and Media': 55.99, 'Health and...",None,"[Entertainment and Media, Health and Fitness, ..."
3,106601471.0,10.6900,18.705,45.5975,34.9075,97.95875,150.3200,220.1350,NaN,NaN,NaN,{'Electronics and Entertainment Retail': 178.7...,{'Electronics and Entertainment Retail': 234.9...,{'Electronics and Entertainment Retail': 309.9...,None,"[Electronics and Entertainment Retail, Enterta..."
4,108481285.0,10.3350,21.980,45.6275,35.2925,98.56625,151.5050,222.0900,{913801979.0: 2185.0},{913801979.0: 2552.5},{913801979.0: 3042.5},"{'Entertainment and Media': 29.73375, 'Health ...","{'Entertainment and Media': 42.78, 'Health and...","{'Entertainment and Media': 60.175, 'Health an...",[913801979.0],"[Entertainment and Media, Health and Fitness, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,995615876.0,25.8750,53.850,108.9750,83.1000,233.62500,358.2750,524.4750,NaN,NaN,NaN,{'Electronics and Entertainment Retail': 126.2...,{'Electronics and Entertainment Retail': 174.9...,{'Electronics and Entertainment Retail': 239.9...,None,"[Electronics and Entertainment Retail, Financi..."
972,996042490.0,15.9900,34.730,93.4750,77.4850,209.70250,325.9300,480.9000,{792845450.0: 1500.0},{792845450.0: 1500.0},{792845450.0: 1500.0},{'Electronics and Entertainment Retail': 237.4...,{'Electronics and Entertainment Retail': 349.9...,{'Electronics and Entertainment Retail': 499.8...,[792845450.0],"[Electronics and Entertainment Retail, Enterta..."
973,998390769.0,15.5600,74.790,145.4825,129.9225,340.36625,535.2500,795.0950,NaN,NaN,NaN,{'Electronics and Entertainment Retail': 219.9...,{'Electronics and Entertainment Retail': 309.9...,{'Electronics and Entertainment Retail': 429.9...,None,"[Electronics and Entertainment Retail, Enterta..."
974,998405607.0,15.9900,35.000,100.0000,84.0100,226.01500,352.0300,520.0500,NaN,NaN,NaN,{'Electronics and Entertainment Retail': 154.9...,{'Electronics and Entertainment Retail': 192.4...,{'Electronics and Entertainment Retail': 242.4...,None,"[Electronics and Entertainment Retail, Enterta..."


In [16]:
anomaly_detection_by_expenditures_df = anomaly_detection_by_expenditures_df.set_index('Account No')

## Risk flagging function- per user transaction

In [17]:
def get_threshold_value(user_data, key, industry=None):
    value = user_data.get(key, 0)
    if isinstance(value, dict):
        return value.get(industry, float('inf'))

def assess_transaction_risk(account_no, amount, third_party_account_no, third_party_name, anomaly_detection_df):
    if amount >= 0:  # Skip payments
        return "Not Applicable"
    
    # Convert the amount to positive for comparison
    amount = abs(amount)
    risk_level = "No Risk"
    user_data = anomaly_detection_df.loc[account_no].to_dict()
    industry_category = tx_df_industry_mapping(third_party_name)
    
    # Initialize interactions as empty lists to ensure they're iterable
    user_user_interaction = user_data.get('user_user_interaction', []) or []
    user_industry_interaction = user_data.get('user_industry_interaction', []) or []
    
    # Check if third_party_account_no is not None and is in user_user_interaction
    if third_party_account_no and third_party_account_no in user_user_interaction:
        thresholds = [
            get_threshold_value(user_data, 'Third_Party_Account_Threshold_1', third_party_account_no),
            get_threshold_value(user_data, 'Third_Party_Account_Threshold_2', third_party_account_no),
            get_threshold_value(user_data, 'Third_Party_Account_Threshold_3', third_party_account_no)
        ]
    # Check if industry_category is determined and in user_industry_interaction
    elif industry_category and industry_category in user_industry_interaction:
        thresholds = [
            get_threshold_value(user_data, 'industry_threshold_1_IQR', industry_category),
            get_threshold_value(user_data, 'industry_threshold_2_IQR', industry_category),
            get_threshold_value(user_data, 'industry_threshold_3_IQR', industry_category)
        ]
    else:
        # Use general thresholds if no specific interaction is found
        thresholds = [user_data['Overall_Threshold_1_IQR'], user_data['Overall_Threshold_2_IQR'], user_data['Overall_Threshold_3_IQR']]
    
    if amount > thresholds[2]:
        risk_level = "High Risk"
    elif amount > thresholds[1]:
        risk_level = "Medium Risk"
    elif amount > thresholds[0]:
        risk_level = "Low Risk"
    
    return risk_level




risk_levels = [
    assess_transaction_risk(
        row['Account No'], row['Amount'], row.get('Third Party Account No'), row['Third Party Name'],
        anomaly_detection_by_expenditures_df
    ) for index, row in testing_transactions.iterrows()
]

testing_transactions['risk_level'] = risk_levels


testing_transactions

,Date,Timestamp,Account No,Balance,Amount,Third Party Account No,Third Party Name,Datetime,industry_category,risk_level
110276,26/06/2023,05:13:00,547010927.0,14786.249167,-7.74,NaN,Amazon,2023-06-26 05:13:00,Entertainment and Media,No Risk
113678,01/07/2023,00:00:00,986774156.0,1666.714362,-939.00,NaN,LBG,2023-07-01 00:00:00,Financial Services,No Risk
102644,09/06/2023,03:36:00,183888402.0,13918.517600,-36.25,NaN,JustEat,2023-06-09 03:36:00,Takeaway,No Risk
98481,01/06/2023,08:12:00,822718326.0,6732.194082,-9.13,NaN,Tesco,2023-06-01 08:12:00,Retail - Clothing and Fashion,No Risk
135544,31/07/2023,23:59:00,494156986.0,-312.797554,-7.99,NaN,Disney,2023-07-31 23:59:00,Entertainment and Media,No Risk
...,...,...,...,...,...,...,...,...,...,...
55831,30/03/2023,12:53:00,635679314.0,-976.793829,-96.68,NaN,Tesco,2023-03-30 12:53:00,Retail - Clothing and Fashion,No Risk
188677,14/10/2023,10:39:00,421881811.0,3080.374643,-36.11,NaN,Brilliant Brushes,2023-10-14 10:39:00,"Retail - Crafts, Hobbies and Books",No Risk
96461,31/05/2023,23:59:00,593191200.0,1764.992250,-18.99,NaN,PureGym,2023-05-31 23:59:00,Health and Fitness,No Risk
182205,02/10/2023,12:43:00,165691047.0,98.783000,-148.00,NaN,Matalan,2023-10-02 12:43:00,Retail - Clothing and Fashion,Low Risk


In [18]:
risk_level_counts = testing_transactions['risk_level'].value_counts()
print(risk_level_counts)


No Risk           21181
Not Applicable      974
Low Risk            507
Medium Risk         144
High Risk           114
Name: risk_level, dtype: int64


In [19]:
# Filter for Low Risk transactions
low_risk_transactions = testing_transactions[testing_transactions['risk_level'] == 'Low Risk']

# Filter for Medium Risk transactions
medium_risk_transactions = testing_transactions[testing_transactions['risk_level'] == 'Medium Risk']

# Filter for High Risk transactions
high_risk_transactions = testing_transactions[testing_transactions['risk_level'] == 'High Risk']

# Filter for Not Applicable transactions
not_applicable_transactions = testing_transactions[testing_transactions['risk_level'] == 'Not Applicable']


In [20]:
low_risk_transactions.describe()

,Account No,Balance,Amount,Third Party Account No
count,5.070000e+02,507.000000,507.000000,5.000000e+00
mean,5.385782e+08,4762.501254,-143.912485,6.331765e+08
std,2.582068e+08,5877.481206,196.870426,3.463330e+08
min,1.085632e+08,-2270.016241,-3730.000000,1.161696e+08
25%,2.980002e+08,764.781065,-201.425000,4.786511e+08
50%,5.553934e+08,2819.431510,-114.980000,7.110456e+08
75%,7.648210e+08,6611.187986,-58.950000,8.737486e+08
max,9.984056e+08,40553.574098,-6.050000,9.862676e+08


In [21]:
medium_risk_transactions.describe()

,Account No,Balance,Amount,Third Party Account No
count,1.440000e+02,144.000000,144.000000,6.000000e+00
mean,5.262492e+08,4405.314303,-179.324028,3.606148e+08
std,2.603688e+08,5449.180988,346.504005,2.910571e+08
min,1.084813e+08,-1011.007700,-2920.000000,1.213637e+08
25%,3.090734e+08,660.811022,-215.242500,1.831860e+08
50%,5.158135e+08,2316.486911,-112.470000,2.342035e+08
75%,7.294269e+08,6865.387815,-49.575000,4.489869e+08
max,9.942144e+08,34880.613071,-5.900000,8.871566e+08


In [22]:
high_risk_transactions

,Date,Timestamp,Account No,Balance,Amount,Third Party Account No,Third Party Name,Datetime,industry_category,risk_level
102428,08/06/2023,11:37:00,604288474.0,3600.401233,-82.75,NaN,Boots,2023-06-08 11:37:00,Pharmacy,High Risk
154410,31/08/2023,10:32:00,407574996.0,1033.070079,-20.64,NaN,Amazon,2023-08-31 10:32:00,Entertainment and Media,High Risk
150152,23/08/2023,12:12:00,973086490.0,8120.943925,-58.07,NaN,Amazon,2023-08-23 12:12:00,Entertainment and Media,High Risk
77708,30/04/2023,23:59:00,229502829.0,-1126.524974,-12.99,NaN,SquareOnix,2023-04-30 23:59:00,Technology and Gaming,High Risk
32950,14/02/2023,09:55:00,941285623.0,781.200857,-119.97,NaN,Gamestation,2023-02-14 09:55:00,Electronics and Entertainment Retail,High Risk
...,...,...,...,...,...,...,...,...,...,...
6632,03/01/2023,10:46:00,969804197.0,403.761600,-220.00,345514953.0,NaN,2023-01-03 10:46:00,NaN,High Risk
164421,09/09/2023,08:02:00,823208989.0,1728.477513,-68.23,NaN,Amazon,2023-09-09 08:02:00,Entertainment and Media,High Risk
183874,05/10/2023,12:24:00,604288474.0,2973.166655,-256.19,NaN,Fitted Stitch,2023-10-05 12:24:00,"Retail - Crafts, Hobbies and Books",High Risk
189577,16/10/2023,00:29:00,897714622.0,8360.534088,-71.05,NaN,Amazon,2023-10-16 00:29:00,Entertainment and Media,High Risk


In [23]:
filtered = anomaly_detection_by_expenditures_df.loc[anomaly_detection_by_expenditures_df.index== 229502829.0]
filtered

,0.25,0.5,0.75,IQR,Overall_Threshold_1_IQR,Overall_Threshold_2_IQR,Overall_Threshold_3_IQR,Third_Party_Account_Threshold_1,Third_Party_Account_Threshold_2,Third_Party_Account_Threshold_3,industry_threshold_1_IQR,industry_threshold_2_IQR,industry_threshold_3_IQR,user_user_interaction,user_industry_interaction
Account No,,,,,,,,,,,,,,,
229502829.0,9.0,18.99,62.97,53.97,143.925,224.88,332.82,NaN,NaN,NaN,{'Entertainment and Media': 27.990000000000002...,"{'Entertainment and Media': 39.99, 'Health and...","{'Entertainment and Media': 55.99, 'Health and...",None,"[Entertainment and Media, Health and Fitness, ..."


In [24]:
# Accessing 'user_industry_interaction' for a specific account
user_industry_interaction = filtered.loc[229502829.0, 'industry_threshold_1_IQR']
print(user_industry_interaction)



{'Entertainment and Media': 27.990000000000002, 'Health and Fitness': 18.99, 'Pubs': 12.537500000000001, 'Retail - Clothing and Fashion': 231.2325, 'Retail - Crafts, Hobbies and Books': 87.455, 'Takeaway': 32.595000000000006, 'Technology and Gaming': 7.0}


In [25]:
filtered_tx_df = copied_simulated_tx_df.loc[
    (copied_simulated_tx_df['Account No'] == 229502829.0) & 
    (copied_simulated_tx_df['Third Party Name'].isin(["Mojang Studios", "Xbox", "Blizzard", "SquareOnix"]))
]
filtered_tx_df

,Date,Timestamp,Account No,Balance,Amount,Third Party Account No,Third Party Name,Datetime
1332,01/01/2023,00:00:00,229502829.0,1535.030000,-7.00,NaN,Xbox,2023-01-01 00:00:00
1333,01/01/2023,00:00:00,229502829.0,1522.040000,-12.99,NaN,SquareOnix,2023-01-01 00:00:00
41501,28/02/2023,23:59:00,229502829.0,-658.878324,-5.99,NaN,Mojang Studios,2023-02-28 23:59:00
59937,31/03/2023,23:59:00,229502829.0,-876.725774,-7.00,NaN,Xbox,2023-03-31 23:59:00
59938,31/03/2023,23:59:00,229502829.0,-882.715774,-5.99,NaN,Mojang Studios,2023-03-31 23:59:00
77708,30/04/2023,23:59:00,229502829.0,-1126.524974,-12.99,NaN,SquareOnix,2023-04-30 23:59:00
98337,31/05/2023,23:59:00,229502829.0,-1483.106524,-7.00,NaN,Xbox,2023-05-31 23:59:00
98338,31/05/2023,23:59:00,229502829.0,-1489.096524,-5.99,NaN,Mojang Studios,2023-05-31 23:59:00
98339,31/05/2023,23:59:00,229502829.0,-1496.096524,-7.00,NaN,Xbox,2023-05-31 23:59:00
98340,31/05/2023,23:59:00,229502829.0,-1503.096524,-7.00,NaN,Xbox,2023-05-31 23:59:00


### Trialling with standard deviations

In [ ]:
# Calculate the mean and std for each account
mean_std = expenditures_training_df.groupby('Account No')['Amount'].agg(['mean', 'std'])

# Establishing thresholds based on standard deviations from the mean
mean_std['Threshold_1_STD'] = mean_std['mean'] + mean_std['std'] * 1  # 1 std from the mean
mean_std['Threshold_2_STD'] = mean_std['mean'] + mean_std['std'] * 2  # 2 stds from the mean
mean_std['Threshold_3_STD'] = mean_std['mean'] + mean_std['std'] * 3  # 3 stds from the mean

# Your anomaly detection DataFrame now is mean_std with the standard deviation thresholds
anomaly_detection_by_expenditures_std_df = mean_std

# Renaming the index to 'Account No' if needed
anomaly_detection_by_expenditures_std_df.index.rename('Account No', inplace=True)

anomaly_detection_by_expenditures_std_df


### Feature engineering for anomaly detection: payments

In [ ]:
# Calculating Q1, Q3, and IQR for each account in 'from_totally_fake_account'
quartiles = payments_training_df.groupby('Account No')['Amount'].quantile([0.25, 0.5, 0.75]).unstack()
quartiles['IQR'] = quartiles[0.75] - quartiles[0.25]

thresholds = quartiles.copy()
thresholds['Overall_Threshold_1_IQR'] = quartiles[0.75] + 1.5 * quartiles['IQR']
thresholds['Overall_Threshold_2_IQR'] = quartiles[0.75] + 3 * quartiles['IQR']
thresholds['Overall_Threshold_3_IQR'] = quartiles[0.75] + 5 * quartiles['IQR']

anomaly_detection_by_payments_df = thresholds

anomaly_detection_by_payments_df.index.rename('Account No', inplace=True)

anomaly_detection_by_payments_df